In [1]:
import os
import multiprocessing
from functools import partial

import faiss
import numpy as np
import scipy.linalg as spla
import torch

from tqdm.notebook import tqdm
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [2]:
class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        self.res = faiss.StandardGpuResources()

        self.gpu_index = faiss.index_cpu_to_gpu(
            self.res,
            0,
            self.index
        )
        self.y = y

    def predict(self, X):
        distances, indices = self.gpu_index.search(X.astype(np.float32), k=self.k)
        votes = self.y[indices]
        predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return indices, distances

In [3]:
# basic params

k = 2
n = 3
N = 500000
num_neg = 250000
num_pos = N - num_neg
r = 1.0
max_norm = 0.1

k_neighbors = k

# making the sphere

points_k = np.random.normal(0, 1, size=(num_pos, k))
points_k = r * (points_k / np.linalg.norm(points_k, ord=2, axis=1).reshape(-1, 1))

points_n = np.zeros((N, n))
points_n[num_neg:, :k] = points_k

poca_idx = np.zeros(num_neg, dtype=np.int64) # point-of-closest-approach -- poca
poca_idx[:num_pos] = np.arange(num_pos, dtype=np.int64)
poca_idx[num_pos:] = np.random.choice(np.arange(num_pos), size=num_neg - num_pos, replace=True).astype(np.int64)
poca = points_n[num_neg:][poca_idx] # points of closest approach


knn = FaissKNeighbors(k=k_neighbors + 1)
knn.fit(points_n[num_neg:], y=np.zeros(num_pos, dtype=np.int64))

# nbhrs, dists = knn.predict()
    
var_thresh = 0.99 # threshold of explained variance for selecting normal directions


In [4]:
indices, dists = knn.predict(poca)



In [ ]:
%%time
off_mfld_pts = np.zeros((num_neg, n))
off_mfld_dists = np.zeros(num_neg)

new_poca = np.zeros((num_neg, n))
new_poca_prturb_sizes = np.zeros(num_neg)

# def make_off_mfld_eg(idx, points_n, indices):

def make_perturbed_poca(idx):
    
    global points_n
    global poca
    global indices
    global dists
    global num_neg
    global num_pos
    
    prturb_size = 0
    
    if idx < num_pos:
        # one copy of poca should be unperturbed
        return (idx, poca[idx], prturb_size)
    
    on_mfld_pt = poca[idx]
    nbhr_indices = indices[idx]
    if idx in nbhr_indices:
        nbhr_indices = nbhr_indices[nbhr_indices != idx]
    else:
        nbhr_indices = nbhr_indices[:-1]
    nbhrs = points_n[num_neg:][nbhr_indices]
    nbhr_local_coords = nbhrs - on_mfld_pt
    nbhr_dists = dists[idx]
    
    pca = PCA()
    pca.fit(nbhr_local_coords)
    expl_var = pca.explained_variance_ratio_
    cum_expl_var = np.cumsum(expl_var)
    tmp = np.where(cum_expl_var > var_thresh)[0][0] + 1
    normal_dirs = pca.components_[tmp:] # picking components that explain (1 - var_thresh)
    tangential_dirs = pca.components_[:tmp]
    
    rdm_coeffs = np.random.normal(0, 1, size=tangential_dirs.shape[0])
    delta = np.sum(rdm_coeffs.reshape(-1, 1) * tangential_dirs, axis=0)
    
    prturb_max = np.sqrt(max(nbhr_dists)) # faiss returns square of L2 norm of difference
    prturb_size = np.random.uniform(0, 1) * prturb_max
    delta = (prturb_size / np.linalg.norm(delta, ord=2)) * delta
    
    prturb_poca = on_mfld_pt + delta
    return (idx, prturb_poca, prturb_size)
    
with multiprocessing.Pool(processes=24) as pool:
    results = pool.map(make_perturbed_poca, range(num_neg))
    
for i in range(num_neg):
    new_poca[i] = results[i][1]
    new_poca_prturb_sizes[i] = results[i][2]

In [ ]:
tangential_dirs

In [ ]:
%%time

new_indices, new_dists = knn.predict(new_poca)     

In [ ]:
%%time

def make_off_mfld_eg(idx):
    
    global new_poca
    global points_n
    global new_indices
    global noise_perturbations
    global num_neg
    global num_pos   
    
    on_mfld_pt = new_poca[idx]
    nbhr_indices = new_indices[idx]
    if idx in nbhr_indices:
        nbhr_indices = nbhr_indices[nbhr_indices != idx]
    else:
        nbhr_indices = nbhr_indices[:-1]
        
    nbhrs = points_n[num_neg:][nbhr_indices]
    nbhr_local_coords = nbhrs - on_mfld_pt
    
    pca = PCA()
    pca.fit(nbhr_local_coords)
    expl_var = pca.explained_variance_ratio_
    cum_expl_var = np.cumsum(expl_var)
    tmp = np.where(cum_expl_var > var_thresh)[0][0] + 1
    normal_dirs = pca.components_[tmp:] # picking components that explain (1 - var_thresh)
        
    
    rdm_coeffs = np.random.normal(0, 1, size=normal_dirs.shape[0])
    off_mfld_pt = np.sum(rdm_coeffs.reshape(-1, 1) * normal_dirs, axis=0)
    rdm_norm = np.random.uniform(0, max_norm)
    off_mfld_pt = off_mfld_pt * (rdm_norm / np.linalg.norm(off_mfld_pt))
    off_mfld_pt += on_mfld_pt
    
    
    return (idx, off_mfld_pt, rdm_norm)

with multiprocessing.Pool(processes=24) as pool:
    results = pool.map(make_off_mfld_eg, range(num_neg))


    
for i in range(len(results)):
    off_mfld_pts[i] = results[i][1]
    off_mfld_dists[i] = results[i][2]
    

    
    
    
# for idx in tqdm(range(N // 2)):
#     on_mfld_pt = points_n[idx]
#     nbhr_indices = indices[idx]
#     nbhr_indices = nbhr_indices[nbhr_indices != idx]
#     nbhrs = points_n[nbhr_indices]
#     nbhr_local_coords = nbhrs - on_mfld_pt
    
#     pca = PCA()
#     pca.fit(nbhr_local_coords)
#     expl_var = pca.explained_variance_ratio_
#     cum_expl_var = np.cumsum(expl_var)
#     tmp = np.where(cum_expl_var > var_thresh)[0][0] + 1
#     normal_dirs = pca.components_[tmp:] # picking components that explain (1 - var_thresh)
    
#     rdm_coeffs = np.random.normal(0, 1, size=normal_dirs.shape[0])
#     off_mfld_pt = np.sum(rdm_coeffs.reshape(-1, 1) * normal_dirs, axis=0)
#     rdm_norm = np.random.uniform(0, max_norm)
#     off_mfld_pt = off_mfld_pt * (rdm_norm / np.linalg.norm(off_mfld_pt))
#     off_mfld_dists[idx] = rdm_norm
#     off_mfld_pt += on_mfld_pt
#     off_mfld_pts[idx] = off_mfld_pt
#     break

    
    

In [ ]:
plt.figure(figsize=(4, 4))
# plt.scatter(off_mfld_pts[:, 0], off_mfld_pts[:, 1], s=0.01, alpha=0.5)
plt.scatter(points_n[num_neg:, 0], points_n[num_neg:, 1], s=0.01, alpha=0.5)
idx = 159
# plt.scatter(off_mfld_pts[:, 0], off_mfld_pts[:, 1], color="cyan")

# plt.scatter(off_mfld_pts[idx, 0], off_mfld_pts[idx, 1], color="red")
plt.scatter(points_n[num_neg:][idx, 0], points_n[num_neg:][idx, 1], color="orange")

plt.scatter(points_n[num_neg:][indices[idx], 0], points_n[num_neg:][indices[idx], 1], s=0.1, c="blue")

nbhr_indices = indices[idx]
if poca_idx[idx] in nbhr_indices:
    nbhr_indices = nbhr_indices[nbhr_indices != poca_idx[idx]]
else:
    nbhr_indices = nbhr_indices[:-1]

nbhrs = points_n[num_neg:][nbhr_indices]
local_nbhr_coords = nbhrs - poca[idx]
pca = PCA(n_components=1)
pca.fit(local_nbhr_coords)

normal_dirs = spla.null_space()

colors = ["orange", "green"]
labels = np.round(pca.explained_variance_ratio_, 3)
arrows = list()
for i in range(pca.components_.shape[0]):
    arr = plt.arrow(points_n[idx, 0], points_n[idx, 1], pca.components_[i, 0], pca.components_[i, 1], label=str(labels[i]), color=colors[i])
    arrows.append(arr)
plt.legend(arrows, [str(i) for i in labels])

In [5]:
if n == 3:
    
    data = list()
    
    data.append(
        go.Scatter3d(
            x=points_n[num_neg:, 0],
            y=points_n[num_neg:, 1],
            z=points_n[num_neg:, 2],
            name="S",
            mode='markers',
            marker=dict(
                size=1,
                color="orange",                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.8
            )
        )
    )
    
    idx = 159
    
    data.append(
        go.Scatter3d(
            x=points_n[num_neg:][indices[idx], 0],
            y=points_n[num_neg:][indices[idx], 1],
            z=points_n[num_neg:][indices[idx], 2],
            name="knn",
            mode='markers',
            marker=dict(
                size=1,
                color="blue",                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.8
            )
        )
    )
    
    

    data.append(
        go.Scatter3d(
            x=poca[idx:idx+1, 0],
            y=poca[idx:idx+1, 1],
            z=poca[idx:idx+1:, 2],
            name="S",
            mode='markers',
            marker=dict(
                size=1,
                color="black",                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.8
            )
        )
    )
    
    fig = go.Figure(data=data)
    
    nbhr_indices = indices[idx]
    if poca_idx[idx] in nbhr_indices:
        nbhr_indices = nbhr_indices[nbhr_indices != poca_idx[idx]]
    else:
        nbhr_indices = nbhr_indices[:-1]

    nbhrs = points_n[num_neg:][nbhr_indices]
    local_nbhr_coords = nbhrs - poca[idx]
    pca = PCA(n_components=1)
    pca.fit(local_nbhr_coords)

    tangent_dirs = pca.components_
    normal_dirs = spla.null_space(tangent_dirs).T
    
    tangent_dirs += poca[idx]
    normal_dirs += poca[idx]
    
    x_lines = list()
    y_lines = list()
    z_lines = list()
    
    for vec in tangent_dirs:
        x_lines.append(poca[idx][0])
        x_lines.append(vec[0])
        x_lines.append(None)
        
        y_lines.append(poca[idx][1])
        y_lines.append(vec[1])
        y_lines.append(None)
        
        z_lines.append(poca[idx][2])
        z_lines.append(vec[2])
        z_lines.append(None)
    
    line_traces = go.Scatter3d(
        x=x_lines,
        y=y_lines,
        z=z_lines,
        mode='lines',
        line = dict(width = 2, color = 'rgb(255, 0,0)'))

    fig.add_trace(line_traces)
    
    x_lines = list()
    y_lines = list()
    z_lines = list()
    for vec in normal_dirs:
        x_lines.append(poca[idx][0])
        x_lines.append(vec[0])
        x_lines.append(None)
        
        y_lines.append(poca[idx][1])
        y_lines.append(vec[1])
        y_lines.append(None)
        
        z_lines.append(poca[idx][2])
        z_lines.append(vec[2])
        z_lines.append(None)
        
    line_traces = go.Scatter3d(
        x=x_lines,
        y=y_lines,
        z=z_lines,
        mode='lines',
        line = dict(width = 2, color = 'rgb(0, 255,0)'))

    fig.add_trace(line_traces)
    
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

    fig.write_html("test-3D.html")
    
    # ------------------------------------------------------------------
        
    rdm_coeffs = np.random.normal(0, 1, size=tangent_dirs.shape[0])
    t_pbn = np.sum(rdm_coeffs.reshape(-1, 1) * tangent_dirs, axis=0)
    t_pbn = t_pbn * (0.01 / np.linalg.norm(t_pbn))
    pb_on_mfld = poca[idx] + t_pbn
    
    new_on_mfld_pt_trace = go.Scatter3d(
            x=pb_on_mfld.reshape(1, -1)[:, 0],
            y=pb_on_mfld.reshape(1, -1)[:, 1],
            z=pb_on_mfld.reshape(1, -1)[:, 2],
            name="S_pb",
            mode='markers',
            marker=dict(
                size=1,
                color="magenta",                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.8
            )
        )
    
    new_indices, new_dists = knn.predict(pb_on_mfld.reshape(1, -1))
    
    new_nbhr_indices = new_indices[0]
    if poca[idx] in new_nbhr_indices:
        new_nbhr_indices = new_nbhr_indices[new_nbhr_indices != poca[idx]]
    else:
        new_nbhr_indices = new_nbhr_indices[:-1]
        
    new_nbhrs = points_n[num_neg:][new_nbhr_indices]
    new_nbhr_local_coords = new_nbhrs - pb_on_mfld
    
    new_pca = PCA(n_components=1)
    new_pca.fit(new_nbhr_local_coords)
    
    new_tangent_dirs = new_pca.components_ 
    new_normal_dirs = spla.null_space(new_tangent_dirs).T 
    new_tangent_dirs += pb_on_mfld
    new_normal_dirs += pb_on_mfld
    
    normal_coeffs = np.random.normal(0, 1, size=new_normal_dirs.shape[0])
    normal_pbn = np.sum(normal_coeffs.reshape(-1, 1) * new_normal_dirs, axis=0)
    
    off_mfld_pt = pb_on_mfld + normal_pbn
    
    x_lines = list()
    y_lines = list()
    z_lines = list()
    
    
    for vec in new_tangent_dirs:
        x_lines.append(pb_on_mfld[0])
        x_lines.append(vec[0])
        x_lines.append(None)
        
        y_lines.append(pb_on_mfld[1])
        y_lines.append(vec[1])
        y_lines.append(None)
        
        z_lines.append(pb_on_mfld[2])
        z_lines.append(vec[2])
        z_lines.append(None)
    
    line_traces = go.Scatter3d(
        x=x_lines,
        y=y_lines,
        z=z_lines,
        mode='lines',
        line = dict(width = 2, color = 'rgb(255, 0,0)'))

    fig.add_trace(line_traces)
    
    
    x_lines = list()
    y_lines = list()
    z_lines = list()
    for vec in new_normal_dirs:
        x_lines.append(pb_on_mfld[0])
        x_lines.append(vec[0])
        x_lines.append(None)
        
        y_lines.append(pb_on_mfld[1])
        y_lines.append(vec[1])
        y_lines.append(None)
        
        z_lines.append(pb_on_mfld[2])
        z_lines.append(vec[2])
        z_lines.append(None)
    
    
    
    line_traces = go.Scatter3d(
        x=x_lines,
        y=y_lines,
        z=z_lines,
        mode='lines',
        line = dict(width = 2, color = 'rgb(0, 255,0)'))
    
    fig.add_trace(line_traces)
    
    off_mfld_trace = go.Scatter3d(
        x=[pb_on_mfld[0], off_mfld_pt[0], None],
        y=[pb_on_mfld[1], off_mfld_pt[1], None],
        z=[pb_on_mfld[2], off_mfld_pt[2], None],
        mode="lines",
        line = dict(width = 2, color = "black")
    )
    
    fig.add_trace(off_mfld_trace)
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    
    fig.write_html("test-3D-2.html")

#     colors = ["orange", "green"]
#     labels = np.round(pca.explained_variance_ratio_, 3)
#     arrows = list()
#     for i in range(pca.components_.shape[0]):
#         arr = plt.arrow(points_n[idx, 0], points_n[idx, 1], pca.components_[i, 0], pca.components_[i, 1], label=str(labels[i]), color=colors[i])
#         arrows.append(arr)
#     plt.legend(arrows, [str(i) for i in labels])

In [6]:
normal_dirs

array([[-1.6497022 ,  1.13070007,  0.        ],
       [-0.8248624 ,  0.56533355,  1.        ]])

In [7]:
poca[idx]

array([-0.8248624 ,  0.56533355,  0.        ])

In [8]:
np.linalg.norm(pb_on_mfld)

0.9929539675692383

In [19]:
np.dot(poca[idx], tangent_dirs[0])

1.0000806799343072

In [28]:
np.dot(pca.components_[0], spla.null_space(pca.components_).T[1])

0.0

In [ ]:
pca.components_

In [ ]:
nbhr_indices

In [ ]:
pca.components_

In [ ]:
normal_dirs

In [ ]:
pca.components_.shape

In [ ]:
plt.figure(figsize=(4, 4))


In [ ]:
np.dot(off_mfld_pt - on_mfld_pt, nbhr_local_coords[2])

In [ ]:
rdm_idx = np.random.choice(np.arange(num_neg), 10000, replace=False)
min_true_dists = np.zeros(rdm_idx.shape[0])
dev_frm_on_mfld = np.zeros(rdm_idx.shape[0])
for idx in tqdm(range(rdm_idx.shape[0])):
    true_off_mfld_dists = np.linalg.norm(off_mfld_pts[rdm_idx[idx]] - points_n[num_neg:,:], ord=2, axis=1)
    min_true_dist = np.min(true_off_mfld_dists)
    min_true_dists[idx] = min_true_dist
    min_true_dist_idx = np.argmin(true_off_mfld_dists)
    dev_frm_on_mfld[idx] = np.linalg.norm(points_n[num_neg:][min_true_dist_idx] - new_poca[rdm_idx[idx]])
    
    

In [ ]:
rdm_idx

In [ ]:
plt.hist(min_true_dists - off_mfld_dists[rdm_idx])
plt.xlabel("abs. error between true and approx. distance")
plt.ylabel("count")
plt.title("error in distance for k={},n={} (10k random samples)".format(k, n))
plt.show()

In [ ]:
max(min_true_dists - off_mfld_dists[rdm_idx])

In [ ]:
plt.hist(new_poca_prturb_sizes)

In [ ]:
np.where(np.abs(min_true_dists - off_mfld_dists[rdm_idx]) > 5e-6)

In [ ]:
plt.hist(dev_frm_on_mfld)
plt.xlabel("L2 distance")
plt.ylabel("count")
plt.title("true vs approx. point of approach for k={},n={} (10k random samples)".format(k, n))
plt.show()

In [ ]:
max(dev_frm_on_mfld)

In [ ]:
np.argmin(np.linalg.norm(off_mfld_pts[0] - points_n[:N//2,:], ord=2, axis=1)), \
np.min(np.linalg.norm(off_mfld_pts[0] - points_n[:N//2,:], ord=2, axis=1))

In [ ]:
np.linalg.norm(off_mfld_pts[0] - points_n[:N//2,:], ord=2, axis=1)

In [ ]:
np.linalg.norm(points_n[141727] - points_n[0])

In [ ]:
off_mfld_dists[0]

In [ ]:
tmp = np.random.normal(0, 1, size=(50, 500))
tmp_pca = PCA()
tmp_pca.fit(tmp)


In [ ]:
tmp_pca.components_.shape

In [ ]:
tmp_pca.explained_variance_ratio_

In [6]:
a = np.array(
    [
        [
            [0, 1, 0],
            [1, 0, 0]
        ],
        [
            [0, 1, 0],
            [0, 0, 1]
        ]
    ]
)

In [14]:
list(range(4, 10))

[4, 5, 6, 7, 8, 9]